# Linblad Dissapation

This code demonstrates the effects various Linblad dissapation operators can have on the evolution of a quantum state

This code works by simulating the evolution of an input pure state subject to a driving Hamiltonian and the effects of various dissapators. We are given control over the parameters of the input pure state, the time length of the evolution, the graining of the simulation, the parameters of the Hamiltonian, and the various dissapator amplitudes. We make use of qutip's mesolve for the purpose of simiulation.

To operate the code, run all the scripts in a juypter notebook, adjust the degrees of freedom with the slider bars, and press Run Interact to observe the output

Mathematical Description
===

The general Linblad equation is
$$ \frac{d}{dt} \rho(t) = -i [H, \rho] + \sum_j \gamma_j \bigg( A_j \rho A_j^\dagger - \frac{1}{2} \Big\{ A_j^\dagger A_j, \rho \Big\} \bigg) $$
$$  = -i [H, \rho] + \gamma \mathcal{L}_D[\rho] $$

$\mathcal{L}_D[\rho]$ is the "Linblad contribution" or the "total dissapator". Each $A_j$ is a "disspator" or "Linblad operator." $\gamma_j$ is a real amplitude associated with the contribution of the dissapator indexed by $j$ to the total dissapator. The $\gamma_j$'s may be called "dissapator amplitudes." 

*The Hamiltonian used in this code is:*
$$ H = w \mathbb{1} + \vec{v} \cdot \vec{\sigma} $$

*The dissapators used in the code are:*

$|0\rangle \langle 1| = \sigma^- = (X + i Y)/2$. Acts like amplitude damping

$\sigma^+$. Acts like amplitude ramping

$|0 \rangle \langle 0| = (\mathbb{1}+Z)/2$ "Zero projection operator"

$|1 \rangle \langle 1| = (\mathbb{1}-Z)/2$ "One projection operator"

$X$

$Y$

$Z$


In [6]:
import numpy as np
import qutip as qt
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interact, interactive, fixed, interact_manual, FloatSlider
import ipywidgets as widgets
import mayavi
%matplotlib inline

In [7]:
def SolveAndPlot(alpha, phi, w, vx, vy, vz, T, DeltaT, AmpDamp, AmpRamp, Proj0, Proj1, SX, SY, SZ):
    """
    Function wrapper of the code used for animation
    """
    # time vector
    tvec = np.linspace(0, T, DeltaT)
    
    # calculate \psi_0
    beta = np.sqrt(1-alpha**2)
    psi0 = alpha*qt.basis(2,0) + beta*np.exp(-1j*2*np.pi*phi/360)*qt.basis(2,1)
    
    # define Linblad operators 
    Lops = [ 
        AmpDamp*qt.destroy(2),                   
        AmpRamp*qt.create(2),                    
        Proj0*(qt.qeye(2)+qt.sigmaz())/2,
        Proj1*(qt.qeye(2)-qt.sigmaz())/2,    
        SX*qt.sigmax(),                     
        SY*qt.sigmay(),                   
        SZ*qt.sigmaz()                      
        ]
    
    
    
    # calculate H
    H = w*qt.qeye(2) + vx*qt.sigmax() + vy*qt.sigmay() + vz*qt.sigmaz() 
    
    # solve
    ExpVec = qt.mesolve(H, psi0, tvec, Lops, [qt.sigmax(), qt.sigmay(), qt.sigmaz()])
    PsiVec = qt.mesolve(H, psi0, tvec, Lops, [])
    
    # plot
    fig, ax = plt.subplots()
    ax.plot(ExpVec.expect[0]);
    ax.plot(ExpVec.expect[1]);
    ax.plot(ExpVec.expect[2]);
    ax.set_xlabel('Time');
    ax.set_ylabel('Expectation values');
    ax.legend(("Sigma-X", "Sigma-Y", "Sigma-Z"));
    # ax.set_box_aspect((1, 1, 1)) 
    # ax = a3.Axes3D(plt.figure(figsize=(15,15))) 
    plt.show(fig)

    # Bloch plots 
    b = qt.Bloch()
    nrm = mpl.colors.Normalize(0,T)
    colors = cm.cool(nrm(tvec))
    b.vector_color = list(colors)
    for s in PsiVec.states:
        b.add_states(s)
    b.show()

In [9]:
Alpha = FloatSlider(min=0, max=1, step=0.05)
Phi = FloatSlider(min=0, max=360, step=1)
W = FloatSlider(min=0, max=10, step=0.5)
Vx = FloatSlider(min=0, max=10, step=0.5)
Vy = FloatSlider(min=0, max=10, step=0.5)
Vz = FloatSlider(min=0, max=10, step=0.5)
Ts = FloatSlider(min=1, max=100, step=1)
DeltaTs = FloatSlider(min=1, max=400, step=1)
ampdamp = FloatSlider(min=0, max=1, step=0.05)
ampramp = FloatSlider(min=0, max=1, step=0.05)
proj0 = FloatSlider(min=0, max=1, step=0.05)
proj1 = FloatSlider(min=0, max=1, step=0.05)
sX = FloatSlider(min=0, max=1, step=0.05)
sY = FloatSlider(min=0, max=1, step=0.05)
sZ = FloatSlider(min=0, max=1, step=0.05)
interact_manual(SolveAndPlot, alpha=Alpha, phi=Phi, w=W, vx=Vx, vy=Vy, vz=Vz, T=Ts, DeltaT=DeltaTs, AmpDamp=ampdamp, AmpRamp=ampramp, Proj0=proj0, Proj1=proj1, SX=sX, SY=sY, SZ=sZ)

interactive(children=(FloatSlider(value=0.0, description='alpha', max=1.0, step=0.05), FloatSlider(value=0.0, …

<function __main__.SolveAndPlot(alpha, phi, w, vx, vy, vz, T, DeltaT, AmpDamp, AmpRamp, Proj0, Proj1, SX, SY, SZ)>